<a href="https://colab.research.google.com/github/jorgemunozl/vla-test/blob/main/TEST/twelve_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Twelve Test - Debug just the subtask training.

This test use the same forward pass that mine model, debug here becomes pretty easy.

## Setup

The following cells set up the environment:
1. Clone the XHUMAN repository
2. Install dependencies
3. Authenticate with HuggingFace Hub
4. Import required libraries

In [1]:
# Clone


Cloning into 'XHUMAN'...
remote: Enumerating objects: 2176, done.
remote: Counting objects: 100% (382/382), done.
remote: Compressing objects: 100% (247/247), done.
remote: Total 2176 (delta 249), reused 239 (delta 129), pack-reused 1794 (from 2)
Receiving objects: 100% (2176/2176), 7.87 MiB | 32.63 MiB/s, done.
Resolving deltas: 100% (1392/1392), done.


In [2]:
%cd XHUMAN

/content/XHUMAN


In [3]:
!uv pip install -e .[pi]

Using Python 3.12.12 environment at: /usr
Resolved 215 packages in 17.56s
Prepared 46 packages in 20.93s
Uninstalled 15 packages in 1.19s
Installed 46 packages in 223ms
 + async-lru==2.1.0
 + av==15.1.0
 + comm==0.2.3
 + deepdiff==8.6.1
 - diffusers==0.36.0
 + diffusers==0.35.2
 + draccus==0.10.0
 + evdev==1.9.2
 + faker==40.1.2
 + feetech-servo-sdk==1.0.0
 - huggingface-hub==0.36.0
 + huggingface-hub==0.35.3
 + inquirerpy==0.3.4
 - ipykernel==6.17.1
 + ipykernel==7.1.0
 + jedi==0.19.2
 + json5==0.13.0
 + jsonlines==4.0.0
 + jupyter==1.1.1
 - jupyter-client==7.4.9
 + jupyter-client==8.8.0
 + jupyter-lsp==2.3.0
 + jupyterlab==4.5.2
 + jupyterlab-server==2.28.0
 + lerobot==0.4.3 (from git+https://github.com/huggingface/lerobot.git@0b067df57d21d3a02d6c511f1609172fa39ac29b)
 + mergedeep==1.3.4
 + mypy-extensions==1.1.0
 - notebook==6.5.7
 + notebook==7.5.2
 - nvidia-cudnn-cu12==9.10.2.21
 + nvidia-cudnn-cu12==9.5.1.17
 - nvidia-cusparselt-cu12==0.7.1
 + nvidia-cusparselt-cu12==0.6.3
 - nvi

In [ ]:
from google.colab import userdata
userdata.get('HF_TOKEN')

In [5]:
import time
from contextlib import nullcontext
from typing import Any

import torch
from accelerate import Accelerator
from accelerate.utils import DistributedDataParallelKwargs
from torch.optim import Optimizer

from lerobot.configs import parser
from lerobot.datasets.sampler import EpisodeAwareSampler
from lerobot.datasets.utils import cycle
from lerobot.optim.factory import make_optimizer_and_scheduler
from lerobot.policies.pretrained import PreTrainedPolicy
from lerobot.utils.logging_utils import AverageMeter, MetricsTracker
from lerobot.utils.random_utils import set_seed
from lerobot.utils.train_utils import load_training_state
from lerobot.utils.utils import (
    format_big_number,
    has_method,
    init_logging,
)

from xhuman.policies.factory import make_xhuman_policy, make_xhuman_pre_post_processors
from xhuman.configs.train import TrainPipelineConfigXHUMAN
from xhuman.datasets.factory import make_dataset_xhuman
from xhuman.datasets.utils import split_train_eval_episodes
from xhuman.logger import logger

## Helper Functions

These functions handle dataset loading and policy updates. They are designed to work with distributed training using HuggingFace Accelerate.

In [6]:
def load_dataset(cfg: TrainPipelineConfigXHUMAN, episodes: list[int], is_main_process: bool = True, accelerator: Accelerator | None = None):
    """
    Load the dataset for training and evaluation.
    """
    # Dataset loading synchronization: main process downloads first to avoid race conditions
    cfg.dataset.episodes = episodes

    if is_main_process:
        logger.info("Creating dataset")
        dataset = make_dataset_xhuman(cfg)

    accelerator.wait_for_everyone()

    # Now all other processes can safely load the dataset
    if not is_main_process:
        dataset = make_dataset_xhuman(cfg)

    return dataset

In [7]:
def update_policy(
    train_metrics: MetricsTracker,
    policy: PreTrainedPolicy,
    batch: Any,
    optimizer: Optimizer,
    grad_clip_norm: float,
    accelerator: Accelerator,
    lr_scheduler=None,
    lock=None,
) -> tuple[MetricsTracker, dict]:
    """
    Performs a single training step to update the policy's weights.

    This function executes the forward and backward passes, clips gradients, and steps the optimizer and
    learning rate scheduler. Accelerator handles mixed-precision training automatically.

    Args:
        train_metrics: A MetricsTracker instance to record training statistics.
        policy: The policy model to be trained.
        batch: A batch of training data.
        optimizer: The optimizer used to update the policy's parameters.
        grad_clip_norm: The maximum norm for gradient clipping.
        accelerator: The Accelerator instance for distributed training and mixed precision.
        lr_scheduler: An optional learning rate scheduler.
        lock: An optional lock for thread-safe optimizer updates.

    Returns:
        A tuple containing:
        - The updated MetricsTracker with new statistics for this step.
        - A dictionary of outputs from the policy's forward pass, for logging purposes.
    """
    start_time = time.perf_counter()
    policy.train()

    # Let accelerator handle mixed precision
    with accelerator.autocast():
        loss, output_dict = policy.forward(batch)

    # Use accelerator's backward method
    accelerator.backward(loss)

    # Clip gradients if specified
    if grad_clip_norm > 0:
        grad_norm = accelerator.clip_grad_norm_(policy.parameters(), grad_clip_norm)
    else:
        grad_norm = torch.nn.utils.clip_grad_norm_(
            policy.parameters(), float("inf"), error_if_nonfinite=False
        )

    # Optimizer step
    with lock if lock is not None else nullcontext():
        optimizer.step()

    optimizer.zero_grad()

    # Step through pytorch scheduler at every batch instead of epoch
    if lr_scheduler is not None:
        lr_scheduler.step()

    # Update internal buffers if policy has update method
    if has_method(accelerator.unwrap_model(policy, keep_fp32_wrapper=True), "update"):
        accelerator.unwrap_model(policy, keep_fp32_wrapper=True).update()

    train_metrics.loss = loss.item()
    train_metrics.grad_norm = grad_norm.item()
    train_metrics.lr = optimizer.param_groups[0]["lr"]
    train_metrics.update_s = time.perf_counter() - start_time
    return train_metrics, output_dict

In [9]:
from xhuman.policies.pi05ki.configuration_pi05ki import PI05KIConfig

policy_config = PI05KIConfig(repo_id="none",device="cuda")

## Configuration and Setup

Configure your dataset and policy settings here. The dataset configuration specifies which HuggingFace repository to load, and the policy configuration sets up the PI05 model architecture.

In [37]:
from xhuman.configs.default import LerobotDatasetConfig

dataset_config = LerobotDatasetConfig(
    repo_id="NONHUMAN-RESEARCH/pick-and-place-fruits-v2-test",
)

In [38]:

cfg = TrainPipelineConfigXHUMAN(
    dataset=dataset_config,
    policy=policy_config # Example policy configuration, replace with your actual policy path
)
cfg.validate()

## Training Setup

Initialize the Accelerator for distributed training and set up the training environment. The accelerator automatically handles:
- Multi-GPU training
- Mixed precision training
- Gradient synchronization across processes

In [39]:
# Create Accelerator
# It will automatically detect if running in distributed mode or single-process mode
# We set step_scheduler_with_optimizer=False to prevent accelerate from adjusting the lr_scheduler steps based on the num_processes
# We set find_unused_parameters=True to handle models with conditional computation
ddp_kwargs = DistributedDataParallelKwargs(find_unused_parameters=True)
accelerator = Accelerator(step_scheduler_with_optimizer=False, kwargs_handlers=[ddp_kwargs])

init_logging(accelerator=accelerator)

# Determine if this is the main process (for logging and checkpointing)
is_main_process = accelerator.is_main_process

# Set seed if specified
if cfg.seed is not None:
    set_seed(cfg.seed, accelerator=accelerator)

# Use accelerator's device
device = accelerator.device
torch.backends.cudnn.benchmark = True
torch.backends.cuda.matmul.allow_tf32 = True

In [40]:
DEBUG_MODE = True  # Set to False for full training
DEBUG_MAX_EPISODES = 3  # Use only first N episodes for debugging

# First, get total episodes count (load minimal dataset to check)
if is_main_process:
    temp_dataset = make_dataset_xhuman(cfg)
    total_episodes = temp_dataset.meta.total_episodes
    del temp_dataset
    logger.info(f"Total episodes available: {total_episodes}")
else:
    # For non-main processes, use a reasonable default
    # In practice, this will be synced after main process loads
    total_episodes = 4  # Fallback - adjust if needed

accelerator.wait_for_everyone()

# Limit episodes for debugging
if DEBUG_MODE:
    episodes = list(range(min(DEBUG_MAX_EPISODES, total_episodes)))
    if is_main_process:
        logger.info(f"DEBUG MODE: Using only {len(episodes)} episodes")
else:
    episodes = list(range(total_episodes))

# Split episodes
train_episodes, eval_episodes = split_train_eval_episodes(
    episodes, split_ratio=cfg.split_ratio, seed=42
)

# Load dataset with ONLY train episodes (proper way to filter)
# This uses the load_dataset helper function which sets cfg.dataset.episodes
if is_main_process:
    logger.info(f"Loading train dataset with {len(train_episodes)} episodes")
dataset = load_dataset(cfg, train_episodes, is_main_process=is_main_process, accelerator=accelerator)

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

meta/episodes/chunk-000/file-000.parquet:   0%|          | 0.00/117k [00:00<?, ?B/s]

meta/tasks.parquet:   0%|          | 0.00/2.29k [00:00<?, ?B/s]

info.json: 0.00B [00:00, ?B/s]

stats.json: 0.00B [00:00, ?B/s]

...

data/chunk-000/file-000.parquet:   0%|          | 0.00/1.37M [00:00<?, ?B/s]

[15:52:18] INFO     Total episodes available: 11                                      ]8;id=103955;file:///tmp/ipython-input-3318975260.py\ipython-input-3318975260.py]8;;\:]8;id=412728;file:///tmp/ipython-input-3318975260.py#9\9]8;;\

           INFO     DEBUG MODE: Using only 3 episodes                                ]8;id=560656;file:///tmp/ipython-input-3318975260.py\ipython-input-3318975260.py]8;;\:]8;id=458726;file:///tmp/ipython-input-3318975260.py#21\21]8;;\

           INFO     Loading train dataset with 2 episodes                            ]8;id=234053;file:///tmp/ipython-input-3318975260.py\ipython-input-3318975260.py]8;;\:]8;id=146316;file:///tmp/ipython-input-3318975260.py#33\33]8;;\

           INFO     Creating dataset                                                  ]8;id=935518;file:///tmp/ipython-input-1989669429.py\ipython-input-1989669429.py]8;;\:]8;id=571858;file:///tmp/ipython-input-1989669429.py#9\9]8;;\

In [14]:
# Create policy
if is_main_process:
    logger.info("Creating policy")
policy = make_xhuman_policy(
    cfg=cfg.policy,
    ds_meta=dataset.meta,
)

accelerator.wait_for_everyone()

[15:46:35] INFO     Creating policy                                                   ]8;id=631262;file:///tmp/ipython-input-3499831122.py\ipython-input-3499831122.py]8;;\:]8;id=27824;file:///tmp/ipython-input-3499831122.py#3\3]8;;\

processor_config.json:   0%|          | 0.00/253 [00:00<?, ?B/s]

processing_action_tokenizer.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/physical-intelligence/fast:
- processing_action_tokenizer.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


tokenizer_config.json:   0%|          | 0.00/322 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.00 [00:00<?, ?B/s]

## Dataset and Model Information

Display metadata about the loaded dataset and model. This includes:
- Total number of episodes and frames
- Model parameter counts
- Effective batch size (accounting for distributed training)

In [41]:
# Display dataset metadata and model configuration
if is_main_process:
    from pprint import pprint

    print("=" * 80)
    print("DATASET METADATA")
    print("=" * 80)
    print(f"\nDataset Repository: {dataset.repo_id}")
    print(f"Total Episodes: {dataset.meta.total_episodes}")
    print(f"Training Episodes: {len(train_episodes)}")
    print(f"Number of Frames: {dataset.num_frames:,}")
    print(f"Number of Episodes (loaded): {dataset.num_episodes}")


    print("\n" + "=" * 80)

DATASET METADATA

Dataset Repository: NONHUMAN-RESEARCH/pick-and-place-fruits-v2-test
Total Episodes: 11
Training Episodes: 2
Number of Frames: 2,924
Number of Episodes (loaded): 2



In [42]:
processor_kwargs = {}
postprocessor_kwargs = {}

if (cfg.policy.pretrained_path and not cfg.resume) or not cfg.policy.pretrained_path:
    processor_kwargs["dataset_stats"] = dataset.meta.stats

if cfg.policy.pretrained_path is not None:
    processor_kwargs["preprocessor_overrides"] = {
        "device_processor": {"device": device.type},
        "normalizer_processor": {
            "stats": dataset.meta.stats,
            "features": {**policy.config.input_features, **policy.config.output_features},
            "norm_map": policy.config.normalization_mapping,
        },
    }
    postprocessor_kwargs["postprocessor_overrides"] = {
        "unnormalizer_processor": {
            "stats": dataset.meta.stats,
            "features": policy.config.output_features,
            "norm_map": policy.config.normalization_mapping,
        },
    }

In [43]:
# This cell was removed - autoreload is not needed for production training

In [44]:
# Create processors
preprocessor, postprocessor = make_xhuman_pre_post_processors(
    policy_cfg=cfg.policy,
    pretrained_path=cfg.policy.pretrained_path,
    **processor_kwargs,
    **postprocessor_kwargs,
)

In [45]:
# Create optimizer and scheduler
if is_main_process:
    logger.info("Creating optimizer and scheduler")
optimizer, lr_scheduler = make_optimizer_and_scheduler(cfg, policy)

step = 0  # number of policy updates

# Resume from checkpoint if needed
if cfg.resume:
    step, optimizer, lr_scheduler = load_training_state(cfg.checkpoint_path, optimizer, lr_scheduler)

[15:52:26] INFO     Creating optimizer and scheduler                                  ]8;id=31244;file:///tmp/ipython-input-1795964688.py\ipython-input-1795964688.py]8;;\:]8;id=98246;file:///tmp/ipython-input-1795964688.py#3\3]8;;\

In [46]:
# Print training info
if is_main_process:
    num_learnable_params = sum(p.numel() for p in policy.parameters() if p.requires_grad)
    num_total_params = sum(p.numel() for p in policy.parameters())
    logger.info(f"Output dir: {cfg.output_dir}")
    logger.info(f"Steps: {cfg.steps} ({format_big_number(cfg.steps)})")
    logger.info(f"Dataset frames: {dataset.num_frames} ({format_big_number(dataset.num_frames)})")
    logger.info(f"Dataset episodes: {dataset.num_episodes}")
    num_processes = accelerator.num_processes
    effective_bs = cfg.batch_size * num_processes
    logger.info(f"Effective batch size: {cfg.batch_size} x {num_processes} = {effective_bs}")
    logger.info(f"Learnable params: {num_learnable_params} ({format_big_number(num_learnable_params)})")
    logger.info(f"Total params: {num_total_params} ({format_big_number(num_total_params)})")

[15:52:27] INFO     Output dir: outputs/train/2026-01-21/15-52-01_pi05_ki             ]8;id=27824;file:///tmp/ipython-input-4288334519.py\ipython-input-4288334519.py]8;;\:]8;id=588508;file:///tmp/ipython-input-4288334519.py#5\5]8;;\

           INFO     Steps: 100000 (100K)                                              ]8;id=571412;file:///tmp/ipython-input-4288334519.py\ipython-input-4288334519.py]8;;\:]8;id=439898;file:///tmp/ipython-input-4288334519.py#6\6]8;;\

           INFO     Dataset frames: 2924 (3K)                                         ]8;id=848749;file:///tmp/ipython-input-4288334519.py\ipython-input-4288334519.py]8;;\:]8;id=911527;file:///tmp/ipython-input-4288334519.py#7\7]8;;\

           INFO     Dataset episodes: 2                                               ]8;id=732052;file:///tmp/ipython-input-4288334519.py\ipython-input-4288334519.py]8;;\:]8;id=443143;file:///tmp/ipython-input-4288334519.py#8\8]8;;\

           INFO     Effective batch size: 8 x 1 = 8                                  ]8;id=800581;file:///tmp/ipython-input-4288334519.py\ipython-input-4288334519.py]8;;\:]8;id=352944;file:///tmp/ipython-input-4288334519.py#11\11]8;;\

           INFO     Learnable params: 3616757520 (4B)                                ]8;id=376417;file:///tmp/ipython-input-4288334519.py\ipython-input-4288334519.py]8;;\:]8;id=888662;file:///tmp/ipython-input-4288334519.py#12\12]8;;\

           INFO     Total params: 3616757520 (4B)                                    ]8;id=45561;file:///tmp/ipython-input-4288334519.py\ipython-input-4288334519.py]8;;\:]8;id=765179;file:///tmp/ipython-input-4288334519.py#13\13]8;;\

In [47]:
# Create dataloader
if hasattr(cfg.policy, "drop_n_last_frames"):
    logger.info(f"Dropping {cfg.policy.drop_n_last_frames} last frames")
    shuffle = False
    sampler = EpisodeAwareSampler(
        dataset.meta.episodes["dataset_from_index"],
        dataset.meta.episodes["dataset_to_index"],
        drop_n_last_frames=cfg.policy.drop_n_last_frames,
        shuffle=True,
    )
else:
    logger.info("Not dropping any frames")
    shuffle = True
    sampler = None



[15:52:28] INFO     Not dropping any frames                                          ]8;id=396922;file:///tmp/ipython-input-4111286182.py\ipython-input-4111286182.py]8;;\:]8;id=82627;file:///tmp/ipython-input-4111286182.py#12\12]8;;\

HERE

In [ ]:
dataloader = torch.utils.data.DataLoader(
    dataset,
    num_workers=cfg.num_workers,
    batch_size=2,
    shuffle=shuffle and not cfg.dataset.streaming,
    sampler=sampler,
    pin_memory=device.type == "cuda",
    drop_last=False,
    prefetch_factor=2 if cfg.num_workers > 0 else None,
)

In [ ]:
# Prepare everything with accelerator
accelerator.wait_for_everyone()
policy, optimizer, dataloader, lr_scheduler = accelerator.prepare(
    policy, optimizer, dataloader, lr_scheduler
)
dl_iter = cycle(dataloader)

policy.train()

# Setup metrics tracking
train_metrics = {
    "loss": AverageMeter("loss", ":.3f"),
    "grad_norm": AverageMeter("grdn", ":.3f"),
    "lr": AverageMeter("lr", ":0.1e"),
    "update_s": AverageMeter("updt_s", ":.3f"),
    "dataloading_s": AverageMeter("data_s", ":.3f"),
}

effective_batch_size = cfg.batch_size * accelerator.num_processes
train_tracker = MetricsTracker(
    effective_batch_size,
    dataset.num_frames,
    dataset.num_episodes,
    train_metrics,
    initial_step=step,
    accelerator=accelerator,
)

HERE

## Training Loop

The main training loop iterates through batches, performs forward/backward passes, and updates the policy weights.

**Note**: The loop below runs for 6 steps as an example. For full training, replace with:
```python
while step < cfg.steps:
    # ... training code ...
```

Metrics are logged at intervals specified by `cfg.log_freq`.

In [48]:
# Training initialization
# This logs the start of training and shows how many episodes will be used
if is_main_process:
    logger.info("Start offline training on a fixed dataset")
    logger.info(f"Train episodes: {len(train_episodes)}")
    logger.info(f"Total training steps: {cfg.steps}")

[15:52:38] INFO     Start offline training on a fixed dataset                         ]8;id=648564;file:///tmp/ipython-input-2557974599.py\ipython-input-2557974599.py]8;;\:]8;id=928463;file:///tmp/ipython-input-2557974599.py#4\4]8;;\

           INFO     Train episodes: 2                                                 ]8;id=738797;file:///tmp/ipython-input-2557974599.py\ipython-input-2557974599.py]8;;\:]8;id=72933;file:///tmp/ipython-input-2557974599.py#5\5]8;;\

           INFO     Total training steps: 100000                                      ]8;id=303445;file:///tmp/ipython-input-2557974599.py\ipython-input-2557974599.py]8;;\:]8;id=83667;file:///tmp/ipython-input-2557974599.py#6\6]8;;\

In [50]:
from torch.utils.data import Dataset

class SmartSubset(Dataset):
    def __init__(self, dataset, indices):
        self.dataset = dataset
        self.indices = indices

    def __getitem__(self, idx):
        return self.dataset[self.indices[idx]]

    def __len__(self):
        return len(self.indices)

    def __getattr__(self, name):
        # This is the magic part:
        # If the code asks for 'meta', 'fps', etc., and this class doesn't have it,
        # it automatically looks inside the original dataset.
        return getattr(self.dataset, name)

# --- USAGE ---
# Use SmartSubset instead of torch.utils.data.Subset
debug_subset = SmartSubset(dataset, range(0, 50))

# Now create your loader normally
train_dataloader = torch.utils.data.DataLoader(
    debug_subset,
    batch_size=2,
    shuffle=True,
    num_workers=0,
    drop_last=True
)

In [51]:
dl_iter = iter(train_dataloader)

In [52]:
train_metrics = {
    "loss": AverageMeter("loss", ":.3f"),
    "grad_norm": AverageMeter("grdn", ":.3f"),
    "lr": AverageMeter("lr", ":0.1e"),
    "update_s": AverageMeter("updt_s", ":.3f"),
    "dataloading_s": AverageMeter("data_s", ":.3f"),
}

effective_batch_size = cfg.batch_size * accelerator.num_processes
train_tracker = MetricsTracker(
    effective_batch_size,
    dataset.num_frames,
    dataset.num_episodes,
    train_metrics,
    initial_step=step,
    accelerator=accelerator,
)

# Call Subtask Processor
Important track the index for loss mask

In [53]:
frames = next(dl_iter)

In [54]:
batch = preprocessor.subtask(frames)

In [55]:
batch.keys()

dict_keys(['action', 'next.reward', 'next.done', 'next.truncated', 'info', 'action_is_pad', 'task', 'index', 'task_index', 'episode_index', 'observation.images.left', 'observation.images.top', 'observation.images.right', 'observation.state', 'observation.language.tokens', 'observation.language.attention_mask'])

In [59]:
batch["COMPLEMENTARY.DATA"]

KeyError: 'COMPLEMENTARY.DATA'

# Test loss calculation

In [58]:
tokens_id = "observation.language.tokens"
mask_id = "observation.language.attention_mask"

images, img_masks = policy._preprocess_images(batch)
tokens, masks = batch[tokens_id], batch[mask_id]
noise = None
time = None
actions = policy.prepare_action(batch)
# Processor should have a method to get the subtask tokenized
# List of tensors
subtasks_tokenized = [torch.randint(0, 131, (13,)), torch.randint(0, 131, (10,))]


In [67]:
subtasks_tokenized = [torch.randint(0, 131, (13,)), torch.randint(0, 131, (10,))]

Artifical Subtask

In [66]:
tokens[0].shape

torch.Size([200])

In [ ]:
# subtasks_tokenized = batch["observation.subtask"]

In [ ]:
subtask_str = policy._detokenize_subtask(subtasks_tokenized[0])
subtask_str

In [62]:
tokens_str = policy._detokenize_subtask(tokens)
tokens_str

'Task: pick and place the fruits in the basket. Subtask:'

In [69]:
        tokens = tokens.clone()
        original_seq_len = tokens.shape[1]

        pad_positions = (tokens == 0).int().argmax(dim=1)
        # If no padding token found, set to sequence length
        no_pad_mask = (tokens == 0).any(dim=1)
        pad_positions = torch.where(
            no_pad_mask, pad_positions, tokens.shape[1]
        )

In [70]:
token_loss_mask = torch.zeros_like(
            tokens, dtype=torch.bool, device=tokens.device
        )

In [71]:
# Get actual lengths for each subtask (for masking)
list_len_subtask = [t.shape[0] for t in subtasks_tokenized]

In [72]:
        for i in range(tokens.shape[0]):
            pad_idx = pad_positions[i].item()
            discrete_len = list_len_subtask[i]
            subtask_tokens = subtasks_tokenized[i].to(
                device=tokens.device, dtype=tokens.dtype
            )

            # Check if discrete actions fit in the sequence
            if pad_idx + discrete_len > tokens.shape[1]:
                # Truncate discrete actions if they don't fit
                available_space = tokens.shape[1] - pad_idx
                if available_space > 0:
                    subtask_tokens = subtask_tokens[:available_space]
                    discrete_len = available_space
                else:
                    # No space available, skip this sample's discrete actions
                    continue

            # Insert discrete action tokens at padding position
            tokens[i, pad_idx:pad_idx + discrete_len] = subtask_tokens
            # Mark these positions for loss computation
            token_loss_mask[i, pad_idx:pad_idx + discrete_len] = True
            # Mark discrete action positions as valid (True) in masks
            masks[i, pad_idx:pad_idx + discrete_len] = True

In [73]:
        prefix_embs, prefix_pad_masks, prefix_att_masks = policy.model.embed_prefix(
            images, img_masks, tokens, masks,
        )
        prefix_att_masks = prefix_att_masks.clone()

        num_image_embs = prefix_embs.shape[1] - tokens.shape[1]

        # Causal masking for discrete actions
        for i in range(tokens.shape[0]):
            pad_idx = pad_positions[i].item()
            discrete_len = list_len_subtask[i]

            if pad_idx + discrete_len <= tokens.shape[1]:
                discrete_action_start_idx = num_image_embs + pad_idx
                discrete_action_end_idx = (
                    discrete_action_start_idx + discrete_len
                )

                if discrete_action_end_idx <= prefix_att_masks.shape[1]:
                    prefix_att_masks[
                        i, discrete_action_start_idx:discrete_action_end_idx
                    ] = 1

In [83]:
prefix_embs

torch.Size([968, 2048])

In [88]:
prefix_np = prefix_embs[0].detach().to("cpu").numpy()

In [89]:
import pandas as pd
import numpy as np

df = pd.DataFrame(prefix_np)

# Save to CSV
df.to_csv('my_matrix.csv', index=False)

# You can now find 'my_matrix.csv' in the Files folder icon on the left sidebar

In [79]:
num_image_embs

768

In [80]:
tokens.shape

torch.Size([2, 200])

In [75]:
from xhuman.policies.pi05ki.modeling_pi05ki import make_att_2d_masks

In [1]:
import torch.nn.functional as F

att_2d_masks = make_att_2d_masks(prefix_pad_masks, prefix_att_masks)
position_ids = torch.cumsum(prefix_pad_masks, dim=1) - 1
att_2d_masks_4d = policy.model._prepare_attention_masks_4d(att_2d_masks)

def forward_func(prefix_embs, att_2d_masks_4d, position_ids):
    (prefix_out, _), _ = policy.model.paligemma_with_expert.forward(
        attention_mask=att_2d_masks_4d,
        position_ids=position_ids,
        past_key_values=None,
        inputs_embeds=[prefix_embs, None],
        use_cache=False,
        adarms_cond=[None, None],
    )
    return prefix_out

prefix_out = policy.model._apply_checkpoint(
    forward_func, prefix_embs, att_2d_masks_4d, position_ids
)

# prefix_out shape: [B, prefix_seq_len, hidden_dim]
def lm_head_func(prefix_out):
    return policy.model.paligemma_with_expert.paligemma.lm_head(prefix_out)

logits = policy.model._apply_checkpoint(lm_head_func, prefix_out)

num_image_embs = prefix_embs.shape[1] - tokens.shape[1]
lang_logits = logits[:, num_image_embs:, :]

# Shift for next-token prediction: predict tokens[i+1] from logits[i]
# shift_logits shape: [B, lang_seq_len-1, vocab_size]
shift_logits = lang_logits[:, :-1, :].contiguous()
# shift_labels shape: [B, lang_seq_len-1]
shift_labels = tokens[:, 1:].contiguous()

# Shift mask to align with shifted logits/labels
# shift_mask shape: [B, lang_seq_len-1]
shift_mask = token_loss_mask[:, 1:].contiguous()

# Flatten for cross-entropy computation
# flat_logits shape: [B*(lang_seq_len-1), vocab_size]
flat_logits = shift_logits.view(-1, shift_logits.size(-1))
# flat_labels shape: [B*(lang_seq_len-1)]
flat_labels = shift_labels.view(-1)
# flat_mask shape: [B*(lang_seq_len-1)]
flat_mask = shift_mask.view(-1)

# Compute loss per token (only on valid positions)
# ce_loss_per_token shape: [B*(lang_seq_len-1)]
ce_loss_per_token = F.cross_entropy(
    flat_logits, flat_labels, reduction='none'
)

NameError: name 'make_att_2d_masks' is not defined

In [77]:
# Apply mask and compute mean per sample
# masked_ce shape: [B, lang_seq_len-1]
masked_ce = (ce_loss_per_token * flat_mask).view(
    tokens.shape[0], -1
)
# ce_loss shape: [B] - average loss per sample over valid tokens
ce_loss = masked_ce.sum(dim=1) / (shift_mask.sum(dim=1) + 1e-8)

In [78]:
print(f"Cross entropy loss shape: {ce_loss.shape}")
print(f"Cross entropy loss: {ce_loss}")
print(f"Mean loss: {ce_loss.mean().item():.4f}")

Cross entropy loss shape: torch.Size([2])
Cross entropy loss: tensor([13.5479, 13.9289], device='cuda:0', grad_fn=<DivBackward0>)
Mean loss: 13.7384
